In [ ]:
# adapt classification algorithm to be score predictor. How to do this? 
# First step - read and load data 

In [1]:
# pytorch setup and import data
import torch
import torchtext

from torch.utils.data import Dataset, DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

# define nn
from torch import nn
from torch.nn import functional as F

# train
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc
from torch.optim import Adam

# evaluate performance
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
import matplotlib.pyplot as plt
import numpy as np


In [18]:
import gzip
import json
import spacy
import string
import pandas as pd
import re

from collections import Counter
from joblib import Parallel, delayed

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn import metrics

## Read Functions

raw data has been read in, cleaned and saved to csv. I assume this is not best practice? More suitable to build the cleaning into the dataloader?

In [17]:
# def parse(path):
#     g = gzip.open(path, 'rb')
#     for l in g:
#         yield json.loads(l)

# def getDF(path):
#     i = 0
#     df = {}
#     for d in parse(path):
#         df[i] = d
#         i += 1
#     return pd.DataFrame.from_dict(df, orient='index')

## Clean and preprocess functions

In [19]:
# def cleaner(df):
#     "Extract relevant text from DataFrame using a regex"
#     # Regex pattern for only alphanumeric, hyphenated text with 3 or more chars
#     pattern = re.compile(r"[A-Za-z0-9\-]{3,50}")
#     df['clean'] = df['reviewText'].str.findall(pattern).str.join(' ')
#     return df

# def lemmatize_pipe(doc):
#     lemma_list = [str(tok.lemma_).lower() for tok in doc
#                   if tok.is_alpha and tok.text.lower() not in stopwords] 
#     return lemma_list

# def preprocess_pipe(texts):
#     preproc_pipe = []
#     for doc in nlp.pipe(texts, batch_size=20):
#         preproc_pipe.append(lemmatize_pipe(doc))
#     return preproc_pipe

# def chunker(iterable, total_length, chunksize):
#     return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

# def flatten(list_of_lists):
#     "Flatten a list of lists to a combined list"
#     return [item for sublist in list_of_lists for item in sublist]

# def process_chunk(texts):
#     preproc_pipe = []
#     for doc in nlp.pipe(texts, batch_size=20):
#         preproc_pipe.append(lemmatize_pipe(doc))
#     return preproc_pipe

# def preprocess_parallel(texts, chunksize=100):
#     executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
#     do = delayed(process_chunk)
#     tasks = (do(chunk) for chunk in chunker(texts, len(df_clean), chunksize=chunksize))
#     result = executor(tasks)
#     return flatten(result)

In [35]:
# n_limit = 1143470 # 100000
# set_chunksize = 500

# stopwords = text.ENGLISH_STOP_WORDS
# nlp = spacy.load("en_core_web_sm")

# raw_df = getDF('../../../../data/Grocery_and_Gourmet_Food_5.json.gz')
# raw_df = raw_df.dropna(subset = ["reviewText"])

# df_limit = raw_df.head(n_limit)
# df_clean = cleaner(df_limit)

# df_clean['text'] = preprocess_parallel(df_clean['clean'], chunksize=set_chunksize)

# df_clean = df_clean[["overall", "text"]]

# # save df_clean as csv
# df_clean.to_csv('df_clean.csv', index=False)
# df_clean.head(3)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,overall,text
0,5.0,"[adverse, comment]"
1,5.0,"[gift, college, student]"
2,5.0,"[like, strong, tea, little, strong]"


In [38]:
tokenizer = get_tokenizer("basic_english")

# df_clean=pd.read_csv("df_clean.csv")

vocab = build_vocab_from_iterator(list(df_clean['text']), min_freq=1, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

In [39]:
len(vocab)

95765

In [40]:
tokens = tokenizer("Hello how are you?, Welcome to CoderzColumn!!")
indexes = vocab(tokens)

tokens, indexes

(['hello',
  'how',
  'are',
  'you',
  '?',
  ',',
  'welcome',
  'to',
  'coderzcolumn',
  '!',
  '!'],
 [4189, 0, 41063, 11466, 0, 0, 2122, 1220, 0, 0, 0])

In [41]:
vocab["<UNK>"] ## Coderzcolumn word is mapped to unknown as it's new and not present in vocabulary

0

In [ ]:
# create test and train dataset from df_clean &/or df_clean with list of lists... 
# from torchtext.data.functional import to_map_style_dataset - converts iterable dataset to map_style_dataset

## Questions for Dan: 
* Is it silly to perform nlp cleaning as a separate step, save to csv and then load that in 'MyDataset'? Some smarter approach?
* I feel like it would be useful to be able to readily edit transforms. 
* 

* put two batches through, 1 in each format and compare
* move heavy lifting cleaning steps into data_loader
* could compare categories to continuous. 

In [88]:
class MyDataset(Dataset):
 
    def __init__(self,indices):
        self.df_clean = pd.read_csv("df_clean.csv").iloc[indices]

    #     x=price_df.iloc[:,0:8].values
    #     y=price_df.iloc[:,8].values

#         self.x_train = df_clean["text"]
#         y = df_clean["overall"].values

# #         self.x_train=torch.tensor(x,dtype=torch.int32)# Don't convert text to tensor yet 
#         self.y_train=torch.tensor(y, dtype=torch.int32)

    def __len__(self):
        return len(self.df_clean["overall"])

    def __getitem__(self,idx):
        return self.df_clean.iloc[idx, 1], self.df_clean.iloc[idx, 0]

In [89]:
# Create train, validate and test
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# labels
dataY = df_clean["overall"]

# features
dataX_dummy = range(len(df_clean["text"])) 

# train is now 75% of the entire data set
index_train, index_test, y_train, y_test = train_test_split(dataX_dummy, dataY, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
index_val, index_test, y_val, y_test = train_test_split(index_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify=y_test) 


train_set = MyDataset(index_train)
val_set = MyDataset(index_val)
test_set = MyDataset(index_test)

In [90]:
print(train_set[0])
print(val_set[0])
print(test_set[0])

("['smooth', 'orange', 'juice', 'doesn', 'bitter', 'taste', 'process', 'do', 'buy']", 5.0)
("['great', 'travel', 'nausea']", 5.0)
("['try', 'keep', 'heavy', 'cream', 'freeze', 'small', 'need', 'freeze', 'destroy', 'taste', 'texture', 'opinion', 'detest', 'buy', 'half', 'pint', 'use', 'half', 'ultimately', 'throw', 'rest', 'heavy', 'cream', 'powder', 'add', 'just', 'tablespoon', 'creamy', 'soup', 'sauce', 'just', 'extra', 'bit', 'creaminess', 'won', 'beat', 'fresh', 'cream', 'need', 'bit', 'cream', 'recipe']", 5.0)


In [91]:
# This will/(may?) look different because text is already a list of words... Still want to convert to tokens. 
# Should I normalize Y/scores data prior? I don't think that would matter?

max_words = 25

def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y, dtype=torch.int32)

train_loader = DataLoader(train_dataset, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset , batch_size=1024, collate_fn=vectorize_batch)

In [96]:
for X, Y in train_loader:
    print(X.shape, Y.shape)
    print("---")
    print(Y)
    print("---")
    print(X)
    break

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/torch/utils/data/datapipes/iter/combining.py:249: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


torch.Size([1024, 25]) torch.Size([1024])
---
tensor([3, 3, 3,  ..., 1, 2, 3], dtype=torch.int32)
---
tensor([[    0,  3396, 27527,  ..., 11367, 27527,    56],
        [ 1868, 62375,   400,  ..., 72484,     0,  3319],
        [    0,  1220,  8240,  ..., 10388,     0,  6711],
        ...,
        [ 1687,     0,   743,  ..., 23459,  1149,  6146],
        [ 1209,     0,     0,  ...,     0, 21961,     0],
        [ 3069,  8019,     0,  ..., 16088,  7524, 68073]], dtype=torch.int32)


In [95]:
# Change classifier to be score predictor. 

In [ ]:
embed_len = 50
hidden_dim = 50
n_layers=1

right now the final layer is len(target_classes...) 


In [ ]:
embed_len = 50
hidden_dim = 50
n_layers=1

class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(target_classes))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings, torch.randn(n_layers, len(X_batch), hidden_dim))
        return self.linear(output[:,-1])